In [1]:
import argoverse
import torch

In [ ]:
from argoverse.map_representation.map_api import ArgoverseMap

In [13]:
a=torch.randn(2,3,4)
a.shape
a

tensor([[[ 1.7241e-01,  7.3474e-01,  6.7025e-01, -2.5457e+00],
         [ 1.2746e-01, -9.7692e-01, -1.0448e-01, -1.5189e-01],
         [ 7.5036e-01,  3.1237e-04, -1.0763e+00,  1.8481e-01]],

        [[ 9.3107e-01, -1.1606e-01, -6.0820e-01, -8.8039e-01],
         [-1.7798e+00, -1.3101e+00, -1.4513e+00, -5.2073e-01],
         [ 1.6815e+00,  7.3596e-01,  1.5130e+00, -8.7353e-01]]])

In [12]:
b=torch.randn(2,3,1)
b

tensor([[[ 0.1456],
         [ 0.1413],
         [ 1.8430]],

        [[ 1.4236],
         [ 1.9804],
         [-2.3153]]])

In [14]:
a*b
a.mul

tensor([[[ 2.5105e-02,  1.0698e-01,  9.7594e-02, -3.7067e-01],
         [ 1.8005e-02, -1.3800e-01, -1.4758e-02, -2.1456e-02],
         [ 1.3829e+00,  5.7571e-04, -1.9836e+00,  3.4060e-01]],

        [[ 1.3255e+00, -1.6523e-01, -8.6583e-01, -1.2533e+00],
         [-3.5246e+00, -2.5945e+00, -2.8741e+00, -1.0312e+00],
         [-3.8931e+00, -1.7040e+00, -3.5031e+00,  2.0225e+00]]])

In [1]:
import torch 
a=torch.tensor([[1,2,3],[4,5,6]])
b=torch.tensor([[2,3,4],[5,6,7]])
c=torch.mul(a,b)
print('a:',a)
print('b:',b)
print('c:',c)


a: tensor([[1, 2, 3],
        [4, 5, 6]])
b: tensor([[2, 3, 4],
        [5, 6, 7]])
c: tensor([[ 2,  6, 12],
        [20, 30, 42]])


In [5]:
a.shape

torch.Size([2, 3])

In [8]:
a
b=a.sum(dim=-1)
b.shape

torch.Size([2])

In [ ]:
b,c=a.chunk(2,dim=-1)
b.shape,c.shape

In [ ]:
a=torch.randn(4)
a.shape
b,c=a.chunk(2,dim=-1)
b.shape,c.shape

In [ ]:
print(a,b,c)

In [ ]:
a=[1,2,3]
a[:1]

# 2023.6.15

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
b=pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]])
b

In [ ]:
a=[1,2,4]
b[b[1].isin(a)]
# b[1].isin(a)
# b[1]
type(b[0])

In [ ]:
df=pd.read_csv("/mnt/home/husiyuan/code/argo_my/ARGO1/data/train/data/1.csv") # Dataframe格式所以df

timestamps=list(np.sort(df['TIMESTAMP'].unique())) #list:一辆车被记录的所有帧，df['TIMESTAMP']是Series, Hash table-based unique去重以后unique()返回ndarray
# timestamps
historical_timestamps=timestamps[:20] # ?why 可改 只取前20帧从0-19
# historical_timestamps
historical_df=df[df['TIMESTAMP'].isin(historical_timestamps)]   # 返回boolean series，然后选择true的行
# historical_df


actor_ids=list(historical_df['TRACK_ID'].unique()) # 前20帧的无重复track_id
# print(len(actor_ids))

df=df[df['TRACK_ID'].isin(actor_ids)] # 是series
print(historical_df.shape)
print(df.shape)
# num_nodes=len(actor_ids)

df.head()

In [ ]:

for a,b in df.groupby('TRACK_ID'):
    print(a)
    print(b)
    # print('\n')

In [ ]:
av_df = df[df['OBJECT_TYPE'] == 'AV'].iloc
av_index = actor_ids.index(av_df[0]['TRACK_ID'])
av_df.__sizeof__() # 56 因为50帧+6个属性
av_df[49]

In [ ]:
df.head()

In [ ]:
df['CITY_NAME'].values[0]

# 2023.6.18

In [ ]:
import torch

In [ ]:
torch.randn(3,4)

In [ ]:
condition=torch.zeros(3,2)
condition

In [ ]:
print(condition[:,1].shape)
print(condition[:,1].unsqueeze(-1).shape)

In [ ]:

condition[:,1:].unsqueeze(-1).shape

In [ ]:
from itertools import permutations
print(list(permutations(range(3),2)))

In [ ]:
int(('134'))

# 2023.6.19

In [ ]:
import os
import torch

In [ ]:
os.path.splitext("../../../data/forecasting_sample_v1.1/forecasting_sample/data/")[0]

In [ ]:
a=torch.randn(2,3,4)
a.size()

# 2023.6.21

## 2: Define a LightningModule

In [1]:
import os 
from torch import optim,nn,utils,Tensor
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
import pytorch_lightning as pl
import tensorboard

[2023-06-21 06:02:48,324] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
encoder=nn.Sequential(nn.Linear(28*28,64),nn.ReLU(),nn.Linear(64,3))
decoder=nn.Sequential(nn.Linear(3,64),nn.ReLU(),nn.Linear(64,28*28))

In [3]:
# define the LightningModule 把train包进来了！！
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)

        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)  # 太牛逼了！！！
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

# init the autoencoder 就是model+train+optimizer
autoencoder = LitAutoEncoder(encoder, decoder)

## 3: Define a dataset

In [4]:
# setup data
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

## 4: Train the model

The Lightning Trainer automates 40+ tricks including:

In [5]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = pl.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/mnt/home/husiyuan/miniconda3/envs/hivt/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1579: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(
2023-06-21 05:48:16.679108: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-21 05:48:16.721067: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 05:48:17.

Training: 0it [00:00, ?it/s]

## 5: Use the model

In [7]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=99.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

# embed 4 fake images!
fake_image_batch = Tensor(4, 28 * 28)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[ 1.5403e+30,  1.2544e+32, -1.8449e+32],
        [-9.4261e+30,  1.1482e+31,  1.3236e+31],
        [ 9.9755e-02,  3.2822e-02, -1.1311e-01],
        [ 9.6513e-02,  3.0934e-02, -1.1387e-01]], grad_fn=<AddmmBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡


## 6: Visualize training

In [13]:
!tensorboard --logdir . --port=6008

2023-06-21 05:55:47.521858: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-21 05:55:48.276855: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/mnt/home/husiyuan/.local/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /mnt/home/husiyuan/.local/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/mnt/home/husiyuan/.local/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /mnt/home/husiyuan/.local/lib/python3.8/site-packages/tens

## 7: Supercharge training

Enable advanced training features using Trainer arguments. These are state-of-the-art techniques that are automatically integrated into your training loop without changes to your code.

In [5]:
# train on 4 GPUs
trainer =pl.Trainer(
    devices=4,
    accelerator="gpu",
 )

# train 1TB+ parameter models with Deepspeed/fsdp
# 分布式并行-4——PyTorch数据并行DP、DDP、FSDP原理
trainer = pl.Trainer(
    devices=4,
    accelerator="gpu",
    strategy="deepspeed_stage_2",
    precision=16
 )

# 20+ helpful flags for rapid idea iteration
trainer = pl.Trainer(
    max_epochs=10,
    min_epochs=5,
    overfit_batches=1
 )

# # access the latest state of the art techniques
# trainer = pl.Trainer(callbacks=[StochasticWeightAveraging(...)])

/mnt/home/husiyuan/miniconda3/envs/hivt/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:849: UserWarning: You requested multiple GPUs but did not specify a backend, e.g. `Trainer(strategy="dp"|"ddp"|"ddp2")`. Setting `strategy="ddp_spawn"` for you.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


MisconfigurationException: `Trainer(strategy='deepspeed')` or `Trainer(accelerator='deepspeed')` is not compatible with an interactive environment. Run your code as a script, or choose one of the compatible backends: dp, ddp_spawn, ddp_sharded_spawn, tpu_spawn. In case you are spawning processes yourself, make sure to include the Trainer creation inside the worker function.

# 2023.6.27

In [2]:
import torch.nn as nn
import torch

In [4]:
w = torch.Tensor(3, 5)
nn.init.normal_(w)

tensor([[-1.0442,  0.0721, -0.3050,  0.4946, -1.3777],
        [ 0.2024, -1.4048,  0.9374, -1.5055, -0.0323],
        [-0.2134,  2.1043,  0.7709,  0.2246, -0.3878]])

In [4]:
from typing import Optional
import torch

In [46]:
def value(a:Optional[torch.Tensor]):
    return a

In [53]:
b=value([torch.randn(2,3)])

In [48]:
def give(a):
    print(a)
    print(type(a))

In [58]:
give(*b.shape)

AttributeError: 'list' object has no attribute 'shape'

In [59]:
*b

SyntaxError: can't use starred expression here (2754346888.py, line 1)

# 2023/7/15

In [2]:
import numpy as np

In [19]:
a=np.array([[1,2],[3,4]])
a.shape
a

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (2881932770.py, line 1)

In [14]:
np.cumsum(-a[0, 1::-1, :], axis=0)

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

# 2023.7.19 debugpy